In [ ]:
# Importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from datetime import datetime
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

This pre-processing part is mainly based on the notebook by Laura Lewis
https://nbviewer.jupyter.org/github/L-Lewis/Airbnb-neural-network-price-prediction/blob/master/Airbnb-price-prediction.ipynb#Categorical-features

In [ ]:
data_original = pd.read_csv("listings_june_2019_all.csv")

In [ ]:
print(data_original.ndim)
print(data_original.shape)
# 106 columns

In [ ]:
pd.set_option('display.max_columns', len(data_original.columns)) 
pd.set_option('display.max_rows', 106)

In [ ]:
# Show all columns
data_original.head()

In [ ]:
# On the first sight, we might delete columns, that: 
# 1) contains the same information throughout the dataset
# 2) contains text data - this is out f the scope of this analysis, therefore will be dropped

In [ ]:
cols_to_drop = ['listing_url', 'scrape_id', 'last_scraped', 'name', 'summary', 'space', 'description', 'neighborhood_overview',
                'notes', 'transit', 'access', 'interaction', 'house_rules', 'thumbnail_url', 'medium_url', 'picture_url',
                'xl_picture_url', 'host_id', 'host_url', 'host_name', 'host_about', 'host_thumbnail_url',
                'host_picture_url', 'host_verifications', 'calendar_last_scraped']

data = data_original.drop(cols_to_drop, axis=1)

In [ ]:
data.isna().sum()

In [ ]:
drop_missing = ["host_acceptance_rate", "neighbourhood_group_cleansed", "square_feet", "weekly_price" ,"monthly_price",
                "license","jurisdiction_names"]
data = data.drop(drop_missing, axis = 1)

In [ ]:
# Location data:

neighborhood = ["street", "neighbourhood","neighbourhood_cleansed"]

for i in neighborhood:
    print(i, data[i].unique())

# These 4 columns above seems to state similar information, therefore, will be deleted, except for 1 representative
# --> checking the missing data

print(data[neighborhood].isna().sum())# neighbourhood_cleansed will be used

# As we work only with data from Prague, columns such as "city", "state" or market can be dropped, they are same for all,
# We add also the three column from above

to_drop_location = ['zipcode', 'city', 'state', 'market', 'smart_location',
                    'country_code', 'country', 'is_location_exact',"street", "neighbourhood"]

data = data.drop(to_drop_location, axis=1)


In [ ]:
# Check latitude, longitude, as it is crucial for identifying new features
data.latitude.isna().sum()
data.longitude.isna().sum()

In [ ]:
data.columns

In [ ]:
# Columns about number of nights:
# These columns seem correlated, so we will test that first:
min_max_night = data.loc[:, ['minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm']]

corr = min_max_night.corr()
sns.heatmap(corr, 
            xticklabels=corr.columns.values,
            yticklabels=corr.columns.values)

In [ ]:
# Therefore, all columns but "minimum night" and "maximum night" will be dropped
data = data.drop(['minimum_minimum_nights', 'maximum_minimum_nights', 'minimum_maximum_nights', 
                 'maximum_maximum_nights', 'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm'], axis=1)

In [ ]:
# Check the number of listing that host has,
# It seems that "calculated_host_listings_count" is sum of 
# "calculated_host_listings_count_entire_homes", "calculated_host_listings_count_private_rooms", "calculated_host_listings_count_shared_rooms"

(data.calculated_host_listings_count == (data.calculated_host_listings_count_entire_homes +
                                         data.calculated_host_listings_count_private_rooms + 
                                         data.calculated_host_listings_count_shared_rooms)).unique()

# Therefore, three column on the right side will be dropped

# host_total_listings_count and calculated_host_listings_count seems to be similar column, threfore
# only calculated_host_listings_count will be kept
data = data.drop(['host_total_listings_count',"host_listings_count", 'calculated_host_listings_count_entire_homes', 
                  'calculated_host_listings_count_private_rooms', 'calculated_host_listings_count_shared_rooms'], axis=1)

In [ ]:
data.calculated_host_listings_count.hist() #it seems that more thnan 10k hosts has only 1 listings, which means
# It doesnt this column does not add much new information to the dataset

In [ ]:
# All true/false column will be replaced by 0 and 1
data = data.replace({'f': 0, 't': 1})

# We plot the distribution of numerical and boolean categories
data.hist(figsize=(20,20));

In [ ]:
# From graphs, we can see that some categorical/boolean columns consists only of one category,therefore will be dropped
# Also, columns with obvious majority of one value will be dropped too
data = data.drop(["host_has_profile_pic", "has_availability", "requires_license", "is_business_travel_ready",
                 "require_guest_profile_picture", "require_guest_phone_verification"], axis =1)

### Variable description 

id - the unique id for each listings<br>
experiences_offered - slightly unclear as it does not appear to directly relate to Airbnb Experiences, but this seems to be the main recommended category of travel type, e.g. business <br>
host_since - date that the host first joined Airbnb<br>
host_response_time - average amount of time the host takes to reply to messages<br>
host_response_rate - proportion of messages that the host replies to<br>
host_is_superhost - whether or not the host is a superhost, which is a mark of quality for the top-rated and most experienced hosts, and can increase your search ranking on Airbnb<br>
calculated_host_listings_count - how many listings the host has in total<br>
host_identity_verified - whether or not the host has been verified with id<br>
host_neighborhood - Prague districts, where the host is living <br>
neighbourhood_cleansed - Prague districts the property is in<br>
host_location - district, where host is living <br>
property_type - type of property, e.g. house or flat<br>
room_type - type of listing, e.g. entire home, private room or shared room<br>
accommodates - how many people the property accommodates<br>
bathrooms - number of bathrooms<br>
bedrooms - number of bedrooms<br>
beds - number of beds<br>
bed_type - type of bed, e.g. real bed or sofa-bed<br>
amenities - list of amenities<br>
price - nightly advertised price (the target variable)<br>
security_deposit - the amount required as a security deposit<br>
cleaning_fee - the amount of the cleaning fee (a fixed amount paid per booking)<br>
guests_included - the number of guests included in the booking fee<br>
extra_people - the price per additional guest above the guests_included price<br>
minimum_nights - the minimum length of stay<br>
maximum_nights - the maximum length of stay<br>
calendar_updated - when the host last updated the calendar<br>
availability_30 - how many nights are available to be booked in the next 30 days<br>
availability_60 - how many nights are available to be booked in the next 60 days<br>
availability_90 - how many nights are available to be booked in the next 90 days<br>
availability_365 - how many nights are available to be booked in the next 365 days<br>
number_of_reviews - the number of reviews left for the property<br>
number_of_reviews_ltm - the number of reviews left for the property in the last twelve months<br>
first_review - the date of the first review<br>
last_review - the date of the most recent review<br>
review_scores_rating - guests can score properties overall from 1 to 5 stars<br>
review_scores_accuracy - guests can score the accuracy of a property's description from 1 to 5 stars<br>
review_scores_cleanliness - guests can score a property's cleanliness from 1 to 5 stars<br>
review_scores_checkin - guests can score their check-in from 1 to 5 stars<br>
review_scores_communication - guests can score a host's communication from 1 to 5 stars<br>
review_scores_location - guests can score a property's location from 1 to 5 stars<br>
review_scores_value - guests can score a booking's value for money from 1 to 5 stars<br>
instant_bookable - whether or not the property can be instant booked (i.e. booked straight away, without having to message the host first and wait to be accepted)<br>
cancellation_policy - the type of cancellation policy, e.g. strict or moderate<br>
reviews_per_month - calculated field of the average number of reviews left by guest each month<br>

### Analysis of individual column:

In [ ]:
len(data.columns)

**id** = id is kept as the identification for each variable (for the process of identifying new features)

**experiences_offered** = contains only "none" value, therefore will be dropped

In [ ]:
data.experiences_offered.unique()
data = data.drop("experiences_offered", axis=1)

**host_since**, **first_review**, **last_review** (*date columns*)

In [ ]:
# These date columns will be convert to 
# 1) number of days host has been active
# 2) number of days the listing is offered

# First convert date to date format
data["june_29"] = "2019-06-29" # date of scraping the data from Airbnb.com
data["june_29"]  = pd.to_datetime(data["june_29"], format ="%Y/%m/%d")

data["host_since"] = data["host_since"].astype('str') 
data["host_since"] = pd.to_datetime(data["host_since"], format ="%Y/%m/%d")

data["first_review"] = data["first_review"].astype('str') 
data["first_review"] = pd.to_datetime(data["first_review"], format ="%Y/%m/%d")

data["last_review"] = data["last_review"].astype('str') 
data["last_review"] = pd.to_datetime(data["last_review"], format ="%Y/%m/%d")

# Derive new columns
data["days_being_host"] = (data["june_29"] - data["host_since"]).astype("timedelta64[D]")

# I assume listing being active from its first review
data["days_from_first_review"] = (data["june_29"] - data["first_review"]).astype("timedelta64[D]")

data["days_from_last_review"] = (data["june_29"] - data["last_review"]).astype("timedelta64[D]")

# all dates columns
date_columns = ["host_since", "first_review", "last_review", "june_29"]

# drop the original columns that were replaced by age columns
data = data.drop(date_columns, axis = 1)

**host_response_time**, **host_response_rate**

In [ ]:
print(data.host_response_time.isna().sum())
print(round((data.host_response_time.isna().sum()/len(data))*100, 1))

In [ ]:
print(data.host_response_time.unique())
print()
# here "nan" will be convert to "unknown"
data.host_response_time = data.host_response_time.fillna("unknown")
print(data.host_response_time.value_counts(normalize=True))

In [ ]:
print(data.host_response_rate.isna().sum())
print(round((data.host_response_rate.isna().sum()/len(data))*100, 1))

In [ ]:
# as the response rate seems to say similar information as response time (see the numbers below),
# we will kept only response_time column
print(len(data[data.loc[ :,['host_response_time', 'host_response_rate']].isnull().sum(axis=1) == 2]))
data.host_response_rate = data.host_response_rate.fillna("unknown")
data.host_response_rate.value_counts(normalize=True)

In [ ]:
data = data.drop("host_response_rate", axis = 1)

**host_neighbourhood**, **host_location**

In [ ]:
print(data.host_neighbourhood.isna().sum())
data.host_location.isna().sum()

In [ ]:
# As host_neighbourhood consists lot of missing data and host_location seems to consist similar/but more general information,
# We will use this column (host_location) to create a categorical variable, host_lives_near

In [ ]:
# Creating new binary variable based on host_location column
data["host_location"] = data["host_location"].astype(str)

# initiate the column
data["host_lives_near"] = 0
# if the column contains Praha or Prague (canse insensitively), assign 1, 0 otherwise
data.host_lives_near[data.host_location.str.contains(pat = "Prague", case = False)] = 1
data.host_lives_near[data.host_location.str.contains(pat = "Praha", case = False)] = 1


# only 38 is "unknown", these category will be considered as 0, (new category for 38 listings will not be created)

In [ ]:
data.host_lives_near.value_counts()

In [ ]:
# column host_location is not needed anymore
data = data.drop(["host_location", "host_neighbourhood"], axis = 1)

**days_being_host**,**host_is_superhost**, **host_has_profile_pic**, **host_identity_verified**

In [ ]:
data.isna().sum()

In [ ]:
# these 4 columns with characteristics of host has same number of missing values (15)
# These rows will be dropped
len(data[data.loc[ :,['days_being_host', 'host_is_superhost', 'host_identity_verified'] ].isnull().sum(axis=1) == 3])

data = data.dropna(subset=['days_being_host'])

**neighbourhood_cleansed**, **longitude**, **latitude**

In [ ]:
data.neighbourhood_cleansed.unique() # Values will be converted to Prague 1-22

**property_type**

In [ ]:
data.property_type.value_counts()
# In this case, we can see a lot of different categories, therefore, we will convert them into
# three, "Apartment", "House" and "Other"

In [ ]:
# Replacing categories that are types of houses or apartments
data.property_type = data.property_type.replace({'Serviced apartment': 'Apartment',
                                    'Loft': 'Apartment',
                                    "Condominium" : "Apartment",
                                    'Bungalow': 'House',
                                    'Cottage': 'House',
                                    'Villa': 'House',
                                    'Tiny house': 'House',
                                    'Earth house': 'House',
                                    'Chalet': 'House'})

In [ ]:
data.property_type.value_counts()

In [ ]:
data.loc[~data.property_type.isin(['House', 'Apartment']), 'property_type'] = "Other"

**room_type**, **accommodates**

In [ ]:
print(data.room_type.isna().sum())
print(data.room_type.unique())
# This column seems okay

print(data.accommodates.isna().sum())
print(data.accommodates.unique())
# This column seems okay

**bathrooms**, **bedrooms**, **beds**, **bedtype**

In [ ]:
sleeping = data.loc[:,["bathrooms", "bedrooms", "beds", "bed_type"]]
corr = sleeping.corr()
sns.heatmap(corr, 
            xticklabels=corr.columns.values,
            yticklabels=corr.columns.values)

# number of beds and bedrooms seems correlated

# bedrooms vs beds:
print(data["bedrooms"].corr(data["beds"])) # 0.6 = quite a lot

# check missing values
print(sleeping.isna().sum())

In [ ]:
# due to correlation, we will leave only "bedrooms" variable
data = data.drop("beds", axis = 1)

In [ ]:
print(data.bed_type.value_counts()) # bed_type will be dropped, as majority it has bed_type == Real Bed
data = data.drop("bed_type", axis=1)

In [ ]:
# 13 values of missing data in bathroom columns will be imputed with median of the number of bathrooms
data["bathrooms"]= data["bathrooms"].fillna(data["bathrooms"].median())

**amenities**

In [ ]:
# Create a set of all possible amenities
amenities_list = list(data.amenities)
amenities_list_string = " ".join(amenities_list)
amenities_list_string = amenities_list_string.replace('{', '')
amenities_list_string = amenities_list_string.replace('}', ',')
amenities_list_string = amenities_list_string.replace('"', '')
amenities_set = [x.strip() for x in amenities_list_string.split(',')]
amenities_set = set(amenities_set)
amenities_set

In [ ]:
# From the above, some amenities are more important than others,
# The amenities chosen to be important (or its combination)

24-hour check-in <br>
Air conditioning/central air conditioning <br>
Amazon Echo/Apple TV/DVD player/game console/Netflix/projector and screen/smart TV (i.e. non-basic electronics) <br>
BBQ grill/fire pit/propane barbeque<br>
Balcony/patio or balcony<br>
Beach view/beachfront/lake access/mountain view/ski-in ski-out/waterfront (i.e. great location/views)<br>
Bed linens<br>
Breakfast<br>
Cable TV/TV<br>
Coffee maker/espresso machine<br>
Cooking basics<br>
Dishwasher/Dryer/Washer/Washer and dryer<br>
Elevator<br>
Exercise equipment/gym/private gym/shared gym<br>
Family/kid friendly, or anything containing 'children'<br>
Free parking on premises/free street parking/outdoor parking/paid parking off premises/paid parking on premises<br>
Garden or backyard/outdoor seating/sun loungers/terrace<br>
Host greets you<br>
Hot tub/jetted tub/private hot tub/sauna/shared hot tub/pool/private pool/shared pool<br>
Internet/pocket wifi/wifi<br>
Long term stays allowed<br>
Pets allowed/cat(s)/dog(s)/pets live on this property/other pet(s)<br>
Private entrance<br>
Safe/security system<br>
Self check-in<br>
Smoking allowed<br>
Step-free access/wheelchair accessible, or anything containing 'accessible'<br>
Suitable for events<br>

In [ ]:
data.loc[data['amenities'].str.contains('24-hour check-in'), 'check_in_24h'] = 1
data.loc[data['amenities'].str.contains('Air conditioning|Central air conditioning'), 'air_conditioning'] = 1
data.loc[data['amenities'].str.contains('Amazon Echo|HBO GO|Game console|Netflix|Projector and screen|Smart TV'), 'high_end_electronics'] = 1
data.loc[data['amenities'].str.contains('BBQ grill|Fire pit|Propane barbeque'), 'bbq'] = 1
data.loc[data['amenities'].str.contains('Balcony|balcony|Patio|Terrace'), 'balcony'] = 1
data.loc[data['amenities'].str.contains('Beach view|Beachfront|Lake access|Mountain view|Ski-in/Ski-out|Waterfront'), 'nature_and_views'] = 1
data.loc[data['amenities'].str.contains('Bed linens'), 'bed_linen'] = 1
data.loc[data['amenities'].str.contains('Breakfast'), 'breakfast'] = 1
data.loc[data['amenities'].str.contains('TV|Cabel TV'), 'tv'] = 1
data.loc[data['amenities'].str.contains('Coffee maker|Espresso machine'), 'coffee_machine'] = 1
data.loc[data['amenities'].str.contains('Cooking basics'), 'cooking_basics'] = 1
data.loc[data['amenities'].str.contains('Dishwasher|Dryer|Washer'), 'white_goods'] = 1
data.loc[data['amenities'].str.contains('Elevator'), 'elevator'] = 1
data.loc[data['amenities'].str.contains('Exercise equipment|Gym|gym'), 'gym'] = 1
data.loc[data['amenities'].str.contains('Family/kid friendly|Children|children'), 'child_friendly'] = 1
data.loc[data['amenities'].str.contains('parking'), 'parking'] = 1
data.loc[data['amenities'].str.contains('Garden|Outdoor|Sun loungers|Terrace'), 'outdoor_space'] = 1
data.loc[data['amenities'].str.contains('Host greets you'), 'host_greeting'] = 1
data.loc[data['amenities'].str.contains('Bathtub|Hot tub|Jetted tub|hot tub|Sauna|Pool|pool'), 'hot_tub_sauna_or_pool'] = 1
data.loc[data['amenities'].str.contains('Internet|Pocket wifi|Wifi'), 'internet'] = 1
data.loc[data['amenities'].str.contains('Long term stays allowed'), 'long_term_stays'] = 1
data.loc[data['amenities'].str.contains('Pets|pet|Cat(s)|Dog(s)'), 'pets_allowed'] = 1
data.loc[data['amenities'].str.contains('Private entrance'), 'private_entrance'] = 1
data.loc[data['amenities'].str.contains('Safe|Security system'), 'secure'] = 1
data.loc[data['amenities'].str.contains('Self check-in'), 'self_check_in'] = 1
data.loc[data['amenities'].str.contains('Smoking allowed'), 'smoking_allowed'] = 1
data.loc[data['amenities'].str.contains('Step-free access|Wheelchair|Accessible'), 'accessible'] = 1
data.loc[data['amenities'].str.contains('Suitable for events'), 'event_suitable'] = 1

In [ ]:
data.head()

In [ ]:
# Replacing nulls with zeros for new columns
cols_to_replace_nulls = data.loc[:,'check_in_24h':].columns
data[cols_to_replace_nulls] = data[cols_to_replace_nulls].fillna(0)

# Produces a list of amenity features where one category (true or false) contains fewer than 10% of listings
infrequent_amenities = []
for col in data.loc[:,'check_in_24h':].columns:
    if data[col].sum() < len(data)/10:
        infrequent_amenities.append(col)
print(infrequent_amenities)

In [ ]:
# Dropping infrequent amenity features
data = data.drop(infrequent_amenities, axis=1)

# Dropping the original amenity feature
data = data.drop('amenities', axis=1)

**price**, **cleaning_fee**, **security_deposit**, **extra_people**

In [ ]:
# Changing the data types - numbers:
prices =["price", "cleaning_fee", "security_deposit", "extra_people"]

for column in prices:
    data[column] = data[column].replace({'\$':''}, regex = True).replace({',':''}, regex = True)
    data[column] = data[column].astype(float)
    data[column] = data[column].fillna(0)
    # if there is no cleaning fee/security deposit/ payment for extra people
    # our assumption is: If there is NaN value at the above features
    # it means nothing is paid for that ==> 0.
    
data[prices].dtypes

In [ ]:
data[prices].isna().sum() # for cleaning_fee and security_deposit, missing values means, that there is no cleaning fee or 
# security deposit

In [ ]:
corr = data[prices].corr()
sns.heatmap(data[prices].corr(), 
            xticklabels=corr.columns.values,
            yticklabels=corr.columns.values)
# As we can see, security deposit and cleaning fee is almost not correlated with the dependent variable - price, therefore
# both will be dropped


**guests_included**, **minimum_nights**, **maximum_nights**

In [ ]:
print(data.guests_included.isna().sum())
print(data.minimum_nights.isna().sum())
print(data.maximum_nights.isna().sum())
# All three columns semms okay

**calendar updated**

In [ ]:
data.calendar_updated.value_counts()
# Hosts can update their calendar whenever they want
# For example when they "book" some time, just because they are not able to provide the accommodation, etc.
# Therefor, this variable will be dropped, as it is not adding any information to the model
data = data.drop("calendar_updated", axis = 1)

**availability**

In [ ]:
# Several types of availability occurs in the dataset, obviously correlated with each other
availabilty = ['availability_30', 'availability_60', 'availability_90','availability_365']
avail = data[availabilty]

In [ ]:
# As there is no regulation of Airbnb, we will work with the availability_365
data = data.drop(['availability_30', 'availability_60', 'availability_90'], axis = 1)

**number_of_reviews**, **number_of_reviews_ltm**, **review_per_month**

In [ ]:
no_reviews = data[["number_of_reviews", "number_of_reviews_ltm", "reviews_per_month"]]

In [ ]:
#number_of_reviews is correlated with reviews_per_month and also number of reviews_ltm, 
# therefore, both of the column will be dropped
print(data.number_of_reviews.corr(data.reviews_per_month))
print(data.number_of_reviews.corr(data.number_of_reviews_ltm))
data = data.drop(["number_of_reviews_ltm", "reviews_per_month"], axis = 1)

**review columns**

In [ ]:
# Review columns are very important for 
review_cols = ['review_scores_rating',
       'review_scores_accuracy', 'review_scores_cleanliness',
       'review_scores_checkin', 'review_scores_communication',
       'review_scores_location', 'review_scores_value']

In [ ]:
# These columns has high number of missing values, however, we would loss lot of information when dropping such column
# Reviews are one of the most important aspect when choosing a listing for stay
data[review_cols].isna().sum()

In [ ]:
def bin_column(col, bins, labels):
    """
    Takes in a column name, bin cut points and labels, replaces the original column with a
    binned version
    """
    data[col] = pd.cut(data[col], bins=bins, labels=labels, include_lowest=True)
    data[col] = data[col].astype('str')

In [ ]:
# Checking the distributions of the review ratings columns
variables_to_plot = list(data.columns[data.columns.str.startswith("review_scores") == True])
fig = plt.figure(figsize=(12,8))
for i, var_name in enumerate(variables_to_plot):
    ax = fig.add_subplot(3,3,i+1)
    data[var_name].hist(bins=10,ax=ax)
    ax.set_title(var_name)
fig.tight_layout()
plt.show()

In [ ]:
# Creating a list of all review columns that are scored out of 10
variables_to_plot.pop(0)

In [ ]:
for col in variables_to_plot:
    bin_column(col,
               bins=[0, 8, 9, 10],
               labels=['0-8/10', '9/10', '10/10'])

In [ ]:
for col in variables_to_plot:
    data[col] = data[col].replace("nan","no reviews")

In [ ]:
# Score 0 to 100
bin_column('review_scores_rating',
           bins=[0, 80, 95, 100],
           labels=['0-79/100', '80-94/100', '95-100/100'])
data["review_scores_rating"] = data["review_scores_rating"].replace("nan","no reviews")

**cancellation policy**

In [ ]:
data.cancellation_policy.value_counts()

In [ ]:
# Replace some categories
data.cancellation_policy = data.cancellation_policy.replace({
            'super_strict_30': 'strict_14_with_grace_period',
            'super_strict_60': 'strict_14_with_grace_period',
            'strict': 'strict_14_with_grace_period'})

**days_from_first_review**, **days_from_last_review**

In [ ]:
periods = ['days_being_host', 'days_from_first_review', 'days_from_last_review']

In [ ]:
data[periods].isna().sum() # days being host is OK

In [ ]:
# Distribution of the number of days since first review
data.days_from_first_review.hist(figsize=(15,5), bins=30);

In [ ]:
# Distribution of the number of days since last review
data.days_from_last_review.hist(figsize=(15,5), bins=30);

In [ ]:
# We will make from these features categorical variables
# Binning time since first review
bin_column('days_from_first_review',
           bins=[0, 182, 365, 730, 1460, max(data.days_from_first_review)],
           labels=['0-6 months',
                   '6-12 months',
                   '1-2 years',
                   '2-3 years',
                   '4+ years'])

data["days_from_first_review"] = data["days_from_first_review"].replace("nan","no reviews")

In [ ]:
data.days_from_first_review.value_counts()

In [ ]:
# Binning time since last review
bin_column('days_from_last_review',
           bins=[0, 14, 60, 182, 365, max(data.days_from_last_review)],
           labels=['0-2 weeks',
                   '2-8 weeks',
                   '2-6 months',
                   '6-12 months',
                   '1+ year'])

data["days_from_last_review"] = data["days_from_last_review"].replace("nan","no reviews")

In [ ]:
data.days_from_last_review.value_counts()

### Exploratory Data Analysis

##### Numerical values

In [ ]:
data.describe()

**price**

In [ ]:
data.price.hist(bins = 100, range = (0,250000))

In [ ]:
plt.figure(figsize=(16,4))
data.price.hist(bins=100, range=(0,100000))
plt.margins(x=0)
plt.axvline(13500, color='red', linestyle='--')
plt.title("Airbnb prices in Prague up to 100 000 CZK (~3,780€)", fontsize=18) # exchange rate on 10/11/2020
plt.xlabel("Price (CZK)")
plt.ylabel("Number of listings")
plt.show()

In [ ]:
data.price.describe()

In [ ]:
# The price above the 13500 CZK (which is roughly 500 euros) will be considered
data = data.loc[data["price"]  <= 13500] # loss of 288 listings
data[["price"]].boxplot(figsize = (6,6))

In [ ]:
plt.figure(figsize=(16,4))
data.price.hist(bins=100, range=(0,13500))
plt.margins(x=0)
plt.title("Airbnb prices in Prague up to 13,500 CZK (~500€)", fontsize = 16) # exchange rate on 10/11/2020
plt.xlabel("Price (CZK)")
plt.ylabel("Number of listings")
plt.show()

In [ ]:
# Minimal price considered will be 200 CZK
data = data.loc[data["price"]  >= 200] # 1 listing will be dropped

**calculated_host_listings_count**

In [ ]:
# Show top ten hosts with the most listings
data.calculated_host_listings_count.value_counts()

**Categorical and Binary variables**

In [ ]:
# Prague is divided into 22 city administrative districts, or 57 city districts,
# To avoid as much as possible the curse of dimensionality here, we will consider only the administrative districts
# Source: https://www.czso.cz/csu/xa/administrativni-a-uzemni-cleneni-prahy
data.neighbourhood_cleansed.value_counts()

In [ ]:
data.loc[data['neighbourhood_cleansed'].str.contains('Kunratice'), 'neighbourhood_cleansed'] = "Praha 4"

data.loc[data['neighbourhood_cleansed'].str.contains('Slivenec'), 'neighbourhood_cleansed'] = "Praha 5"

data.loc[data['neighbourhood_cleansed'].str.contains('Suchdol'), 'neighbourhood_cleansed'] = "Praha 6"
data.loc[data['neighbourhood_cleansed'].str.contains('Nebušice'), 'neighbourhood_cleansed'] = "Praha 6"
data.loc[data['neighbourhood_cleansed'].str.contains('Lysolaje'), 'neighbourhood_cleansed'] = "Praha 6"
data.loc[data['neighbourhood_cleansed'].str.contains('Přední Kopanina'), 'neighbourhood_cleansed'] = "Praha 6"

data.loc[data['neighbourhood_cleansed'].str.contains('Troja'), 'neighbourhood_cleansed'] = "Praha 8"
data.loc[data['neighbourhood_cleansed'].str.contains('Ďáblice'), 'neighbourhood_cleansed'] = "Praha 8"
data.loc[data['neighbourhood_cleansed'].str.contains('Dolní Chabry'), 'neighbourhood_cleansed'] = "Praha 8"
data.loc[data['neighbourhood_cleansed'].str.contains('Březiněves'), 'neighbourhood_cleansed'] = "Praha 8"

data.loc[data['neighbourhood_cleansed'].str.contains('Újezd'), 'neighbourhood_cleansed'] = "Praha 11"
data.loc[data['neighbourhood_cleansed'].str.contains('Šeberov'), 'neighbourhood_cleansed'] = "Praha 11"

data.loc[data['neighbourhood_cleansed'].str.contains('Libuš'), 'neighbourhood_cleansed'] = "Praha 12"

data.loc[data['neighbourhood_cleansed'].str.contains('Řeporyje'), 'neighbourhood_cleansed'] = "Praha 13"

data.loc[data['neighbourhood_cleansed'].str.contains('Dolní Počernice'), 'neighbourhood_cleansed'] = "Praha 14"

data.loc[data['neighbourhood_cleansed'].str.contains('Petrovice'), 'neighbourhood_cleansed'] = "Praha 15"
data.loc[data['neighbourhood_cleansed'].str.contains('Štěrboholy'), 'neighbourhood_cleansed'] = "Praha 15"
data.loc[data['neighbourhood_cleansed'].str.contains('Dolní Měcholupy'), 'neighbourhood_cleansed'] = "Praha 15"
data.loc[data['neighbourhood_cleansed'].str.contains('Dubeč'), 'neighbourhood_cleansed'] = "Praha 15"


data.loc[data['neighbourhood_cleansed'].str.contains('Zbraslav'), 'neighbourhood_cleansed'] = "Praha 16"
data.loc[data['neighbourhood_cleansed'].str.contains('Velká Chuchle'), 'neighbourhood_cleansed'] = "Praha 16"
data.loc[data['neighbourhood_cleansed'].str.contains('Lochkov'), 'neighbourhood_cleansed'] = "Praha 16"
data.loc[data['neighbourhood_cleansed'].str.contains('Lipence'), 'neighbourhood_cleansed'] = "Praha 16"

data.loc[data['neighbourhood_cleansed'].str.contains('Zličín'), 'neighbourhood_cleansed'] = "Praha 17"

data.loc[data['neighbourhood_cleansed'].str.contains('Čakovice'), 'neighbourhood_cleansed'] = "Praha 18"

data.loc[data['neighbourhood_cleansed'].str.contains('Vinoř'), 'neighbourhood_cleansed'] = "Praha 19"
data.loc[data['neighbourhood_cleansed'].str.contains('Satalice'), 'neighbourhood_cleansed'] = "Praha 19"


data.loc[data['neighbourhood_cleansed'].str.contains('Klánovice'), 'neighbourhood_cleansed'] = "Praha 21"
data.loc[data['neighbourhood_cleansed'].str.contains('Koloděje'), 'neighbourhood_cleansed'] = "Praha 21"
data.loc[data['neighbourhood_cleansed'].str.contains('Běchovice'), 'neighbourhood_cleansed'] = "Praha 21"


data.loc[data['neighbourhood_cleansed'].str.contains('Benice'), 'neighbourhood_cleansed'] = "Praha 22"
data.loc[data['neighbourhood_cleansed'].str.contains('Královice'), 'neighbourhood_cleansed'] = "Praha 22"
data.loc[data['neighbourhood_cleansed'].str.contains('Kolovraty'), 'neighbourhood_cleansed'] = "Praha 22"

In [ ]:
data.neighbourhood_cleansed.value_counts()

In [ ]:
neighborhood_toplot = data.groupby('neighbourhood_cleansed').price.median()
neighborhood_toplot  = pd.DataFrame(neighborhood_toplot)
neighborhood_toplot= neighborhood_toplot.sort_values("price", ascending=False)
neighborhood_toplot.plot(kind="bar",figsize=(20,5))
plt.title('Median price of Airbnb in Prague districts', fontsize=20)
plt.xlabel('Prague districts', fontsize=13)
plt.ylabel('Median price (CZK)', fontsize=13)
plt.xticks(rotation=0)
plt.show()

### Preparing data for modeling and further procedure

In [ ]:
one_hot_data = pd.get_dummies(data)

In [ ]:
def multi_collinearity_heatmap(df, figsize=(11,9)):
    
    """
    Creates a heatmap of correlations between features in the df. A figure size can optionally be set.
    """
    
    # Set the style of the visualization
    sns.set(style="ticks")

    # Create a covariance matrix
    corr = df.corr()

    # Generate a mask the size of our covariance matrix
    mask = np.zeros_like(corr, dtype=np.bool)
    mask[np.triu_indices_from(mask)] = True

    # Set up the matplotlib figure
    f, ax = plt.subplots(figsize=figsize)

    # Generate a custom diverging colormap
    cmap = sns.diverging_palette(220, 10, as_cmap=True)

    # Draw the heatmap with the mask and correct aspect ratio
    sns.heatmap(corr, mask=mask, cmap=cmap, center=0, square=True, linewidths=.5, cbar_kws={"shrink": .5}, vmax=corr[corr != 1.0].max().max());

In [ ]:
multi_collinearity_heatmap(one_hot_data, figsize=(20,20))

In [ ]:
# Neighborhood categories seems uncorrelated, only with latitude,longitude, which make sense,
# Therefore will be dropped in the future analysis

In [ ]:
multi_collinearity_heatmap(one_hot_data.drop(list(one_hot_data.columns[one_hot_data.columns.str.startswith('neighbourhood_cleansed')]), axis=1), figsize=(25,22))

In [ ]:
# we tak a look on the first 10 columns (around bathrooms, bedrooms)
first_10 = one_hot_data.iloc[:,:10]

In [ ]:
multi_collinearity_heatmap(first_10, figsize=(10,10))

In [ ]:
one_hot_data = one_hot_data.drop(["bedrooms", "guests_included"], axis = 1)

In [ ]:
# Take a look again
multi_collinearity_heatmap(one_hot_data.drop(list(one_hot_data.columns[one_hot_data.columns.str.startswith('neighbourhood_cleansed')]), axis=1), figsize=(25,22))

#### Property and room type multi collinearity

In [ ]:
# Property and room type multi collinearity
room_property = one_hot_data.loc[:, one_hot_data.columns.str.startswith('room_type') | one_hot_data.columns.str.startswith('property_type')]

In [ ]:
multi_collinearity_heatmap(room_property, figsize=(5,5))

In [ ]:
# As we can see, property_type_Other and property_type_Apartment are highly correlated
# As well as room_type_Entire home/apt and room_type_Private room
# property_type_Apartment and room_type_Private_room (one from each category), otherwise we would have dummy variable trap

In [ ]:
data.room_type.value_counts()

In [ ]:
one_hot_data = one_hot_data.drop(["property_type_Apartment", "room_type_Private room"], axis = 1)

#### Reviews columns multicollinearity

In [ ]:
reviews = one_hot_data.iloc[:,66:]

In [ ]:
multi_collinearity_heatmap(reviews, figsize=(20,20))

In [ ]:
# apparently, "noreviews" column are highly correlated (which is make sense as review information missing almost always
# in the same listings), hence these categories will be dropped
reviews_to_drop = one_hot_data.columns[one_hot_data.columns.str.endswith('no reviews')]

In [ ]:
one_hot_data = one_hot_data.drop(reviews_to_drop, axis = 1)

In [ ]:
multi_collinearity_heatmap(reviews, figsize=(10,10))
# High reviewed listing tends to be in high reviewed columns for each category, therefore these columns are highly correlated
# Therefore, we will kept only one category review_score_value as representatives of the others

In [ ]:
review_score_kept = one_hot_data.loc[:, one_hot_data.columns.str.startswith('review_scores_value')]

In [ ]:
reviews_scores_to_drop = one_hot_data.columns[one_hot_data.columns.str.startswith('review_scores')]

In [ ]:
one_hot_data = one_hot_data.drop(reviews_scores_to_drop, axis = 1)

In [ ]:
one_hot_data = pd.concat([one_hot_data,review_score_kept], axis=1, join='inner')

In [ ]:
one_hot_data = one_hot_data.drop("host_response_time_unknown", axis = 1) # correlated with host_response_time_within an hour

In [ ]:
# Lastly, we will drop column review_scores_value_9/10, as it is correlated a lot with 10/10
one_hot_data["review_scores_value_10/10"].corr(one_hot_data["review_scores_value_9/10"])
# Also, we would leave only review_score_10
one_hot_data = one_hot_data.drop("review_scores_value_9/10", axis = 1)

### Linear regression

In [ ]:
# Now we will check the distribution of variables, mainly the dependent variable, price

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score, r2_score

In [ ]:
# Separating X and y
X = one_hot_data.drop('price', axis=1)
y = one_hot_data.price

In [ ]:
# Splitting into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=123) 

In [ ]:
# Create instance of the model, `LinearRegression` function from 
# Scikit-Learn and fit the model on the training data:

linreg = LinearRegression()  
linreg.fit(X_train, y_train) # training 

# Now that the model has been fit we can make predictions by calling 
# the predict command. We are making predictions on the testing set:
val_preds_linreg = linreg.predict(X_test)

# Check the predictions against the actual values by using the R-2 metrics:
print("Validation r2:", round(r2_score(y_test, val_preds_linreg),4))

In [ ]:
one_hot_data.to_csv("airbnb.csv")